In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot

In [ ]:
data = pd.read_csv('/content/drive/My Drive/csv_때리기/input1.csv')
data = np.transpose(data)
loaded = data[:30]

for i in range(2,207+1):
  filename1 = '/content/drive/My Drive/csv_때리기/input{}.csv'.format(i)
  filename2 = '/content/drive/My Drive/csv_쓰다듬기/input_쓰다듬기{}.csv'.format(i)
  data = pd.read_csv(filename1)
  data = np.transpose(data)
  loaded = np.concatenate((loaded, data[:30]), axis=0)

  data = pd.read_csv(filename2)
  data = np.transpose(data)
  loaded = np.concatenate((loaded, data[:30]), axis=0)
  
    
loaded = np.reshape(loaded, (413,30,51))

print(loaded.shape)

(413, 30, 51)


In [ ]:
X = loaded[:,:,:50]
y = loaded[:,:,50]


print(X.shape)
print(y.shape)

(413, 30, 50)
(413, 30)


In [ ]:
trainX = X[:330,:,:]
testX = X[330:,:,:]
trainy = y[:330,:]
testy = y[330:,:]

In [ ]:
def evaluate_model(trainX, trainy, testX, testy):
	# define model
	verbose, epochs, batch_size = 0, 25, 1
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# reshape into subsequences (samples, time steps, rows, cols, channels)
	n_steps, n_length = 2, 15
	trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
	testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
	# define model
	model = Sequential()
	model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
	model.add(Dropout(0.5))
	model.add(Flatten())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy


In [ ]:
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
scores = list()
for r in range(10):
  score = evaluate_model(trainX,trainy,testX,testy)
  score = score * 100.0
  print('#%d: %.3f' % (r+1, score))
  scores.append(score)


summarize_results(scores)

#1: 100.000
#2: 100.000
#3: 100.000
#4: 100.000
#5: 100.000
#6: 100.000
#7: 100.000
#8: 100.000
#9: 100.000
#10: 100.000
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
Accuracy: 100.000% (+/-0.000)
